In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import os
import seaborn as sns

from matplotlib.colors import ListedColormap
from pathlib import Path
from digiforest_analysis.utils import plotting

### Parameters

In [ ]:
ROBOT_VELOCITY_THR = 0.01
SAFETY_VELOCITY_THR = 0.1
FILTERING_WINDOW = "10s" #"25s"
BASE_INVERTED = True
INTERVENTION_TIME_THR = 30

In [ ]:
# Missions
ROOT_PATH = "/media/matias/T7/research/2024-autonomous-legged-forester-autonomy"
missions = {
    "M1": {
        "path": f"{ROOT_PATH}/2023-05-03-13-07-17-mission-evo-m1"
    },
    "M2": {
        "path": f"{ROOT_PATH}/2023-05-04-14-36-05-mission-evo-m2"
    },
    "M3": {
        "path": f"{ROOT_PATH}/2023-05-04-14-50-00-mission-evo-m3"
    },
    "M4": {
        "path": f"{ROOT_PATH}/2023-05-04-19-14-10-mission-evo-m4"
    },
    "M5": {
        "path": f"{ROOT_PATH}/2023-05-05-08-10-22-mission-evo-m5"
    },
    "M6": {
        "path": f"{ROOT_PATH}/2023-10-06-12-47-25-mission-whytam-woods"
    },
    "M7": {
        "path": f"{ROOT_PATH}/2024-02-20-09-40-24-mission-forest-of-dean"
    },
}

output_path = ROOT_PATH

## Matplotlib config

In [ ]:
cm = 1 / 2.54
plot_width = 8.89 * cm
plot_height = 8 * cm

plt.rcParams["font.size"] = 8

### Utils

In [ ]:
from scipy.spatial.transform import Rotation as R

def read_poses_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    # Parse data
    poses = {}
    for ts, x, y, z, qx, qy, qz, qw in zip(
        df.index, df["x"], df["y"], df["z"], df["qx"], df["qy"], df["qz"], df["qw"]
    ):
        poses[f"{ts:.10f}"] = np.eye(4)
        poses[f"{ts:.10f}"][0:3, 3] = np.array([x, y, z])
        poses[f"{ts:.10f}"][0:3, 0:3] = R.from_quat([qx, qy, qz, qw]).as_matrix()

        # TODO: fix base inversion
    
    # Compute integrated distance traveled
    df_diff = df.diff()
    df["lin_dist"] = (df_diff["x"]**2 + df_diff["y"]**2).pow(1.0 / 2).cumsum() 
    df["lin_dist"].at[df.index[0]] =  0.0

    return df, poses

In [ ]:
def read_twist_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    # Correct twist due to base inversion
    if BASE_INVERTED:
        df["vx"] *= -1
        df["vy"] *= -1

    # Speeds
    df["lin_speed"] = (df["vx"] ** 2 + df["vy"] ** 2).pow(1.0 / 2)
    df["ang_speed"] = df["wz"].abs()

    return df

In [ ]:
def read_param_change_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep="first")]

    return df

In [ ]:
import itertools

def flip(items, ncol):
    return itertools.chain(*[items[i::ncol] for i in range(ncol)])

### Read data

In [ ]:
for k, m in missions.items():
    mission_path = m["path"]

    missions[k]["df_state_pose"], _ = read_poses_file(
        os.path.join(mission_path, "states/state_pose_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_state_twist"] = read_twist_file(
        os.path.join(mission_path, "states/state_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_reference_twist"] = read_twist_file(
        os.path.join(mission_path, "states/reference_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    missions[k]["df_operator_twist"] = read_twist_file(
        os.path.join(mission_path, "states/operator_twist_data.csv"),
        base_inverted=BASE_INVERTED,
    )
    # missions[k]["df_local_planner_param"] = read_param_change_file(
    #     os.path.join(mission_path, "states/local_planner_param_data.csv"),
    # )
    # missions[k]["df_rmp_param"] = read_param_change_file(
    #     os.path.join(mission_path, "states/rmp_param_data.csv"),
    # )

### Align time series

In [ ]:
for k, m in missions.items():
    # Extend indices
    joined_indices = (
        missions[k]["df_state_twist"]
        .index.union(missions[k]["df_reference_twist"].index)
        .drop_duplicates()
    )
    joined_indices = joined_indices.union(
        missions[k]["df_operator_twist"].index
    ).drop_duplicates()

    joined_indices = joined_indices.union(
        missions[k]["df_state_pose"].index
    ).drop_duplicates()

    # joined_indices = joined_indices.union(
    #     missions[k]["df_local_planner_param"].index
    # ).drop_duplicates()
    # joined_indices = joined_indices.union(
    #     missions[k]["df_rmp_param"].index
    # ).drop_duplicates()

    # Extend state twist
    missions[k]["df_state_twist"] = missions[k]["df_state_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_state_twist"] = missions[k]["df_state_twist"].interpolate(
        method="index"
    )

    # Extend operator twist
    missions[k]["df_operator_twist"] = missions[k]["df_operator_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_operator_twist"] = missions[k]["df_operator_twist"].interpolate(
        method="index"
    )

    # Extend reference twist (local planner)
    missions[k]["df_reference_twist"] = missions[k]["df_reference_twist"].reindex(
        index=joined_indices
    )
    missions[k]["df_reference_twist"] = missions[k]["df_reference_twist"].interpolate(
        method="index"
    )

    # Extend state poses
    missions[k]["df_state_pose"] = missions[k]["df_state_pose"].reindex(
        index=joined_indices
    )
    missions[k]["df_state_pose"] = missions[k]["df_state_pose"].interpolate(
        method="index"
    )
    missions[k]["df_state_pose"] = missions[k]["df_state_pose"].fillna(method='bfill')

    # missions[k]["df_local_planner_param"] = missions[k][
    #     "df_local_planner_param"
    # ].reindex(index=joined_indices)
    # missions[k]["df_rmp_param"] = missions[k]["df_rmp_param"].reindex(
    #     index=joined_indices
    # )

## Estimate metrics

### Normalize time
To start from 0

In [ ]:
for k, m in missions.items():
    ref_ts = missions[k]["df_state_twist"].index[0]

    missions[k]["df_state_pose"].index = missions[k]["df_state_pose"].index - ref_ts
    missions[k]["df_state_twist"].index = missions[k]["df_state_twist"].index - ref_ts
    missions[k]["df_reference_twist"].index = (
        missions[k]["df_reference_twist"].index - ref_ts
    )
    missions[k]["df_operator_twist"].index = (
        missions[k]["df_operator_twist"].index - ref_ts
    )

### Compute all the data we can extract per mission
This is required for the metrics afterwards

In [ ]:
for k, m in missions.items():
    # Get time and distance traveled
    t = missions[k]["df_state_twist"].index.total_seconds()
    d = missions[k]["df_state_pose"]["lin_dist"]

    # Time in motion
    missions[k]["df_state_twist"]["in_motion"] = (
        missions[k]["df_state_twist"]["lin_speed"] > ROBOT_VELOCITY_THR
    )

    # Interventions from safety operator
    missions[k]["df_state_twist"]["safety_intervention"] = (
        missions[k]["df_operator_twist"]["lin_speed"] > SAFETY_VELOCITY_THR
    ).astype(np.float64)
    missions[k]["df_state_twist"]["safety_intervention"] = missions[k][
        "df_state_twist"
    ]["safety_intervention"].interpolate(method="pad")

    # Filter interventions signal
    filt = t * 0  # Just initial value
    # dilation
    filt = (
        missions[k]["df_state_twist"]["safety_intervention"]
        .rolling(FILTERING_WINDOW, center=True)
        .max()
        > 0
    )
    # erosion
    filt = (1.0 - filt).rolling(FILTERING_WINDOW, center=True).max() > 0
    filt = 1.0 - filt
    filt[-1] = 0
    missions[k]["df_state_twist"]["safety_intervention_filtered"] = filt

    # Find time and distance traveled during each intervention
    # We find the peaks in the original interventions signal, that should correspond to the center
    peaks, properties = signal.find_peaks(filt, distance=1)
    missions[k]["safety_intervention_num"] = len(peaks)

    # We find the left and right edges of the peak
    results_half = signal.peak_widths(
        missions[k]["df_state_twist"]["safety_intervention_filtered"], peaks, rel_height=0.5
    )
    pos_l = results_half[2].astype(int)
    pos_r = results_half[3].astype(int)

    # Intervention time
    missions[k]["intervention_time"] = []
    missions[k]["intervention_distance"] = []
    for l, r in zip(pos_l, pos_r):
        dt = t[r] - t[l]
        missions[k]["intervention_time"].append(dt)

        d_dist = d[r] - d[l]
        missions[k]["intervention_distance"].append(d_dist)

    # Time between interventions
    # We need to do some hacking here to use the intervention indices 
    if (len(pos_l) and len(pos_r)) or len(peaks) == 0:
        pos_auto_l = pos_r
        pos_auto_r = pos_l
        pos_auto_l = np.insert(pos_auto_l, 0, 0, axis=0)
        pos_auto_r = np.insert(pos_auto_r, len(pos_auto_r), len(t) - 1, axis=0)

    # Fill the events
    missions[k]["time_between_interventions"] = []
    missions[k]["distance_between_interventions"] = []
    for l, r in zip(pos_auto_l, pos_auto_r):
        # Fill time between interventions
        dt = t[r] - t[l]
        missions[k]["time_between_interventions"].append(dt)

        # Fill distance between interventions
        d_dist = d[r] - d[l]
        missions[k]["distance_between_interventions"].append(d_dist)
    
    # Fill distance traveled
    missions[k]["distance_traveled"] = d[-1]

### Statistics

In [ ]:
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Mission", "Dist. traveled", "Mission time", "Walk time", "Interv.", "Interv. time", "MTBI", "MDBI", "Autonomy % (by time)", "Autonomy % (by dist)"]

for k, m in missions.items():
    n_interv = len(m['intervention_time'])
    dist_traveled = m["distance_traveled"]

    # Mean distance between interventions
    cumulated_dbi = np.asarray(m['distance_between_interventions']).sum()
    mdbi = np.asarray(m['distance_between_interventions']).mean()
    max_between_dist = np.asarray(m['distance_between_interventions']).max()

    # Mean time between interventions
    mtbi = np.asarray(m['time_between_interventions']).mean()

    import scipy.integrate as integrate
    walk_time = integrate.simpson(
        missions[k]["df_state_twist"]["in_motion"],
        missions[k]["df_state_twist"]["in_motion"].index.total_seconds(),
    ).item()

    interv_time = integrate.simpson(
        missions[k]["df_state_twist"]["safety_intervention_filtered"],
        missions[k]["df_state_twist"]["safety_intervention_filtered"].index.total_seconds(),
    ).item()

    mission_time = missions[k]["df_state_twist"].index.total_seconds()[-1]
    autonomy_perc_by_time = 100 * (1.0 - interv_time / (walk_time))
    autonomy_perc_by_dist = 100 * cumulated_dbi / dist_traveled

    table.add_row([k, dist_traveled, mission_time, walk_time, n_interv, interv_time, mtbi, mdbi, autonomy_perc_by_time, autonomy_perc_by_dist])

table.float_format = ".2"
print(table)


## Plots

### Autonomy over time
This plots the full mission time overlaying the actual intervention events

In [ ]:
from scipy import signal

plot_width = 8.89 * cm
plot_height = 7 * cm

N = len(missions)

fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)

# Axes
ax.set_axisbelow(True)
ax.grid(zorder=-1)
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()

yticks_val = []
yticks_label = []
legend_handles = []
legend_labels = []

for i, (k, v) in enumerate(missions.items()):
    t = missions[k]["df_state_twist"].index.total_seconds()
    d = missions[k]["df_state_pose"]["lin_dist"]
    offset = -i * 0.5

    yticks_val.append(offset)
    yticks_label.append(k)

    # Plot standing time
    line_standing = ax.fill_between(
        t,
        offset - 0.05,
        offset + 0.05,
        linewidth=0,
        color=plotting.gray_palette_str["40"],
        alpha=0.7,
        label="Standing",
        zorder=1,
    )

    # Plot walking time
    in_motion = missions[k]["df_state_twist"]["in_motion"]
    line_walking = ax.fill_between(
        t,
        in_motion * (offset - 0.07),
        in_motion * (offset + 0.07),
        linewidth=0,
        color=plotting.color_palette_str["blue"],
        alpha=1.0,
        label="Walking",
        zorder=1,
    )

    # Plot raw interventions
    line_raw_int = ax.fill_between(
        t,
        missions[k]["df_state_twist"]["safety_intervention"] * (offset - 0.02),
        missions[k]["df_state_twist"]["safety_intervention"] * (offset + 0.02),
        linewidth=0,
        color="k",
        alpha=1.0,
        label="Safety op. action",
        zorder=5,
    )

    # Plot interventions
    filt = missions[k]["df_state_twist"]["safety_intervention_filtered"]
    line_int = ax.fill_between(
        t,
        filt * (offset - 0.1),
        filt * (offset + 0.1),
        linewidth=0,
        color=plotting.color_palette_str["orange"],
        alpha=1.0,
        label="Intervention",
        zorder=4,
    )

    if i == 0:
        legend_handles = [line_standing, line_walking, line_raw_int, line_int]
        legend_labels = [h.get_label() for h in legend_handles]

lgnd = ax.legend(
    legend_handles,
    legend_labels,
    edgecolor=(1, 1, 1, 0),
    framealpha=0.9,
    loc=(0.0, 1.01),
    ncol=2,
)
for line in lgnd.get_lines():
    line.set_sizes([40.0])

# Set grid
# loc = plt.MultipleLocator(30.0)  # this locator puts ticks at regular intervals
# ax.xaxis.set_major_locator(loc)
# ax.yaxis.set_major_locator(loc)
ax.margins(x=0.05)
ax.set_yticks(yticks_val, yticks_label, rotation=0)
ax.set_xlabel("Time [s]")

fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "mission_autonomy_summary.pdf"))
fig.savefig(os.path.join(output_path, "mission_autonomy_summary.png"), dpi=300)

### Intervention distribution (by distance)

In [ ]:
accumulated_interventions = []
list_interventions = []
color_interventions = []
label_interventions = []
for i, (k, m) in enumerate(missions.items()):
    accumulated_interventions.extend(m["distance_between_interventions"])
    list_interventions.append(np.asarray(m["distance_between_interventions"], dtype=int))
    color_interventions.append(plotting.color_palette[i])
    label_interventions.append(k)
accumulated_interventions = np.asarray(accumulated_interventions, dtype=int)

# Plotting settings
plot_width = 8.89 * cm
plot_height = 5 * cm

# Plot
fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)
# Axes
ax.set_axisbelow(True)
# ax.set_aspect("equal")
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()
# Plot
ax.hist(
    list_interventions,
    bins=20,
    cumulative=False,
    density=False,
    stacked=True,
    color=color_interventions,
    label=label_interventions,
    rwidth=0.9,
)
# ax.set_title("Distribution of intervention time")
ax.set_xlabel("Distance between interventions [m]")
ax.set_ylabel("Frequency")

lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0.01, 1.01), ncol=4)
for handle in lgnd.legend_handles:
    try:
        handle.set_sizes([40.0])
    except Exception as e:
        pass

ax.yaxis.set_major_locator(plt.MultipleLocator(4))

fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "distribution_distance_between_interventions.pdf"))

### Time Between Interventions (TBI) Histogram

In [ ]:
accumulated_autonomy = []
list_autonomy = []
color_autonomy = []
label_autonomy = []
for i, (k, m) in enumerate(missions.items()):
    accumulated_autonomy.extend(m["time_between_interventions"])
    list_autonomy.append(np.asarray(m["time_between_interventions"], dtype=int))
    color_autonomy.append(plotting.color_palette[i])
    label_autonomy.append(k)
accumulated_autonomy = np.asarray(accumulated_autonomy, dtype=int)

# Plotting settings
plot_width = 8.89 * cm
plot_height = 5 * cm

# Plot
fig, ax = plt.subplots(
    1,
    1,
    figsize=(plot_width, plot_height),
    constrained_layout=False,
    dpi=300,
    sharex=True,
)
# Axes
ax.set_axisbelow(True)
# ax.set_aspect("equal")
ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
ax.grid(
    which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
)
ax.minorticks_on()
# Plot
ax.hist(
    list_autonomy,
    bins=20,
    cumulative=False,
    density=False,
    stacked=True,
    log=False,
    color=color_autonomy,
    label=label_autonomy,
    rwidth=0.9,
)
# ax.set_title("Distribution of intervention time")
ax.set_xlabel("Time between interventions [s]")
ax.set_ylabel("Frequency")
# ax.set_ylim([0, 16])

lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0.01, 1.01), ncol=4)
for handle in lgnd.legend_handles:
    try:
        handle.set_sizes([40.0])
    except Exception as e:
        pass #print(e)

ax.yaxis.set_major_locator(plt.MultipleLocator(4))
# ax.xaxis.set_major_locator(plt.MultipleLocator(50))
fig.set_tight_layout(True)
fig.savefig(os.path.join(output_path, "time_between_interventions_distribution.pdf"))

In [ ]:
# # Plot
# fig, ax = plt.subplots(
#     1,
#     1,
#     figsize=(plot_width, plot_height),
#     constrained_layout=False,
#     dpi=300,
#     sharex=True,
# )
# # Axes
# ax.set_axisbelow(True)
# # ax.set_aspect("equal")
# ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
# ax.grid(
#     which="minor", color=plotting.gray_palette_str["10"], linestyle=":", linewidth=0.5
# )
# ax.minorticks_on()
# # Plot

# _, bins = np.histogram(np.log10(accumulated_autonomy + 1), bins=10)

# ax.hist(
#     list_autonomy,
#     bins=10**bins,
#     cumulative=False,
#     density=False,
#     stacked=True,
#     log=False,
#     color=color_autonomy,
#     label=label_autonomy,
#     rwidth=0.9,
# )
# # ax.set_title("Distribution of intervention time")
# ax.set_xlabel("Time between interventions [s]")
# ax.set_ylabel("Frequency")
# ax.set_xscale("log")

# lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0.09, 1.01), ncol=5)
# for line in lgnd.get_lines():
#     line.set_sizes([40.0])

# fig.set_tight_layout(True)
# fig.savefig(
#     os.path.join(output_path, "time_between_interventions_distribution_log.pdf")
# )

### Robot velocity

In [ ]:
smoothing_window = "3000ms"
linewidth = 0.75

plot_width = 14 * cm
plot_height = 4 * cm

N = len(missions)


for i, (k, m) in enumerate(missions.items()):
    fig, ax = plt.subplots(
        1,
        1,
        figsize=(plot_width, plot_height),
        constrained_layout=False,
        dpi=300,
        sharex=True,
    )

    # Axes
    ax.set_axisbelow(True)
    # ax.set_aspect("equal")
    ax.grid(which="major", color=plotting.gray_palette_str["20"], linewidth=0.7)
    ax.grid(
        which="minor",
        color=plotting.gray_palette_str["10"],
        linestyle=":",
        linewidth=0.5,
    )
    ax.minorticks_on()

    ax.plot(
        missions[k]["df_state_twist"].index.total_seconds(),
        missions[k]["df_state_twist"]["lin_speed"]
        .rolling(smoothing_window, center=True)
        .mean(),
        label="Robot speed",
        linewidth=linewidth,
        color=plotting.color_palette_str["blue"],
    )
    ax.plot(
        missions[k]["df_reference_twist"].index.total_seconds(),
        missions[k]["df_reference_twist"]["lin_speed"]
        .rolling(smoothing_window, center=True)
        .mean(),
        label="Local planner reference",
        linewidth=linewidth,
        color=plotting.gray_palette_str["60"],
    )

    # if missions[k]["df_state_twist"]["operator_intervention"].any():
    #     ax[i].fill_between(
    #         missions[k]["df_state_twist"]["operator_intervention"].index.total_seconds(),
    #         missions[k]["df_state_twist"]["operator_intervention"].index.total_seconds() * 0,
    #         missions[k]["df_state_twist"]["operator_intervention"],
    #         label="Operator action",
    #         linewidth=1,
    #         linestyle="-",
    #         color=plotting.gray_palette_str["90"],
    #         alpha=1.0,
    #     )

    if missions[k]["df_state_twist"]["safety_intervention_filt"].any():
        ax.fill_between(
            missions[k]["df_state_twist"][
                "safety_intervention_filt"
            ].index.total_seconds(),
            missions[k]["df_state_twist"][
                "safety_intervention_filt"
            ].index.total_seconds()
            * 0,
            missions[k]["df_state_twist"]["safety_intervention_filt"],
            label="Intervention",
            linewidth=0,
            color=plotting.color_palette_str["orange"],
            alpha=0.7,
        )

    # Legends
    lgnd = ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0, 1.03), ncol=3)
    for handle in lgnd.legend_handles:
        try:
            handle.set_sizes([40.0])
        except Exception as e:
            print(e)

    for handle in lgnd.get_lines():
        handle.set_linewidth(1)

    # ax.set_title('Computation Time')
    ax.margins(x=0, y=0)
    ax.set_ylabel("Speed [m/s]")
    ax.set_xlabel("Time [s]")

    # Export
    fig.set_tight_layout(True)
    fig.savefig(os.path.join(output_path, f"mission_velocity_{k.lower()}.pdf"))